# Projekt 2 Webscraping

- I detta projekt skall vi ladda ner sales listan från steams shop och i slutändan spara som csv.

### Delmoment 1
- Importera alla moduler vi behöver
- Definiera några små funktioner vi kommer att använda en hel del:
1. Flatten trycker till våra listor så man blir av med onödiga sublistor
2. Vi kommer att använda oss en hel del av map och jag blev lite trött på att hela tiden måsta konverter till en lista så ja gjorde 2 shortcuts lm för map 

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime
# flattens list 
flatten = lambda f : [item for sublist in f for item in sublist]
# shorthand for list map 
lm = lambda f,l: list(map(f,l)) 
# Dictionary för points systemet
reviewWords = {"Overwhelmingly Positive":10,"Very Positive":9,"Positive":8,"Mostly Positive":7,"Mixed":6,"Mostly Negative":5,"Negative":4,"Very Negative":3,"Overwhelmingly Negative":2,"":0}


### Delmoment 2 
- Vi laddar in datan från steam. Hela logiken är insatt i en while loop för att få info från flera sidor.

- Jag hade till att börja med lite svårt att komma igång med det hela å BeautifulSoup kändes tungt. Men som tur så hade jag gått en kurs i Haskell på hösten vilket jag tyckte var väldigt snärttigt. Och när det visade sig att pythons list comprehension, map, filter och lambda är ganska nära Haskells så va de bara att tuta och köra. 

- Koden består ju till stor del av 2 massiva oneliners, som tur mindes jag att Dennis hade på timmen pratat om \ tecknett för dela på oneliners i flerarader. filteredGames linjen börjar väl vara över 600chars :O

- Själva koden är till stor del som sådan: vi söker efter alla parent element som innehåller allt för ett spel: a tags med klassen = "search_result_row". 
- Sedan loopar vi igenom alla resultat och per film använder vi oss av findAll för att hitta de rätta elementen innuti a taggen
- Sedan kör vi en map funktion på hela datan och per film mappar vi ut den rätta datan och sätter in den i en dictionary som vi appendar till thelist listan. 

- När alla listor är sparade går vi vidare och kör flatten på thelist. Då får vi alla dictionaries, från alla de 5 listorna vi har sparat i thelist, till en stor lista, som inehåller alla de 5 listornas alla dictionaries (filmdata). 

- Största problemen hade jag med att få 1 och 0 på Win,Linux,Mac kolumnerna. Men de var ju lätt löst sen när man väl hade satt in datan i en dataframe. Men ja kämppade nog i flera timmar med att försöka lösa det med en map eller filter där som man mappar in datan i filteredGames variabeln


In [2]:
page = 1
thelist= []
while page <6:
    # Get data
    req = requests.get("https://store.steampowered.com/search/?specials=1&page="+str(page) )
    # Soup data
    soup = BeautifulSoup(req.text,"html.parser")
    # Här tar vi å söker reda på de rätta elementen för att få tag på vår data
                               
                  # Första raden är för namnet
    games = [     [g.findAll("span",{"class":"title"}) \
                  # Review datan 
                  ,g.findAll("span",{"class":"search_review_summary"}) \
                  # Priser
                  ,g.findAll("div",{"class":"search_price"}) \
                  # Mängden rabatt 
                  ,g.findAll("div",{"class":"search_discount"}) \
                  # Release datum
                  ,g.findAll("div",{"class":"search_released"}) \
                  # Följande 3 rader söker efter spans med klassen win linux eller mac
                  ,g.findAll("span",{"class":"win"})
                  ,g.findAll("span",{"class":"linux"}) \
                  ,g.findAll("span",{"class":"mac"}) \
                  # Här är den själva for loopen vi går igenom
                  # Vi tar in alla 'a' taggar med rätt klass
                  ] for g in soup.find_all('a',{"class":"search_result_row"})]
    
    # Här tar vi och vidare filtrerar samt sätter in värden i en lista av dictionaries
    # Pga att vi använder map en hel del så finns värdena oftast inne i en lista 
    # vi använder "".join(list) för att få ut värden från listan
    filteredGames = lm(lambda game: {      "Name":game[0][0].text \
                                           # Dethär va lite kinkyt ti att börja med men de va ju bara att access klassen via [""]
                                           ,"Score":"".join(lm(lambda x : x["data-tooltip-html"].replace("<br>"," "),game[1])) \
                                           # elementet innehåller både original price och discounted price
                                           # För att få ut ena elementet tar vi indexen för det första € märket
                                           # lstrip() för att bli av med trailing whitespaces
                                           ,"Original Price":"".join(lm(lambda x: x.text[0:x.text.index('€')+1].strip() if "€" in x.text else x.text.strip() ,game[2]))\
                                           # replace för att bli av med newlines kunde också ha använt lstrip() här
                                           ,"Discount":game[3][0].text.strip()\
                                           ,"Discounted Price":"".join(lm(lambda x: x.text[x.text.index('€')+1:].strip() if "€" in x.text else x.text.strip() ,game[2]))\
                                           # Dethär orsakade mest gråhår i detta projekte 
                                           # pga av att de e ingen skillna vilket värde som sätts i elsen
                                           # retur värdet blir bara tomt om ifen är False
                                           ,"Win": "".join(lm(lambda x: "1" if "win" in x["class"] else "",game[5])) \
                                           ,"Linux": "".join(lm(lambda x: "1" if "linux" in x["class"] else "",game[6])) \
                                           ,"Mac": "".join(lm(lambda x: "1" if "mac" in x["class"] else "",game[7])) \
                                           ,"Release Date":game[4][0].text \
                                           ,"Time":datetime.datetime.now().strftime("%c")}
                                           # Vår lambda tar emot games variabeln
                                           ,games)
    # appenda våran lista på dictionaries till theList
    thelist.append(filteredGames)
    # increment counter
    page = page+1

# Gör en dataframe av våra dictionaries
theFrame = pd.DataFrame(flatten(thelist))

### Delmoment 3
- Här fixas några saker jag int fick löst/orkade lösa inne i onelinerna
- Vissa saker visade sig vara lättare med DataFrames/Series 
- Vi använder oss pånytt av map denna gång pandas series map funktion. 

In [3]:
# Fyll alla tomma värden med 0
theFrame["Win"] = theFrame["Win"].map(lambda x: 1 if x == "1" else 0)
theFrame["Linux"] = theFrame["Linux"].map(lambda x: 1 if x == "1" else 0)
theFrame["Mac"] = theFrame["Mac"].map(lambda x: 1 if x == "1" else 0)
# Förvandla reviewn till poäng baserat på vilken text sträng det finns
# Vi letar reda på % tecknet för att få ut den relevanta strängen
theFrame["Score"] = theFrame["Score"].map(lambda x: reviewWords[x[0:x.index('%')-3].rstrip()] if len(x)>0 else "")

### Delmoment 4 
- Skriv in datan till en csv fil, använd append mode

In [4]:
theFrame.to_csv("steam_data.csv",mode="a")

In [5]:
theFrame

,Name,Score,Original Price,Discount,Discounted Price,Win,Linux,Mac,Release Date,Time
0,Counter-Strike: Global Offensive,9,Free to Play,,Free to Play,1,1,1,"21 Aug, 2012",Fri Nov 27 12:21:26 2020
1,DOOM Eternal,9,"59,99€",-67%,"19,79€",1,0,0,"19 Mar, 2020",Fri Nov 27 12:21:26 2020
2,The Sims™ 4,9,"39,99€",-88%,"4,79€",1,0,0,"18 Jun, 2020",Fri Nov 27 12:21:26 2020
3,Borderlands 3,9,"59,99€",-67%,"19,79€",1,0,0,"13 Mar, 2020",Fri Nov 27 12:21:26 2020
4,Red Dead Redemption 2,9,"59,99€",-33%,"40,19€",1,0,0,"5 Dec, 2019",Fri Nov 27 12:21:26 2020
...,...,...,...,...,...,...,...,...,...,...
120,Remnant: From the Ashes,9,"39,99€",-50%,"19,99€",1,0,0,"19 Aug, 2019",Fri Nov 27 12:21:30 2020
121,Disco Elysium,9,"39,99€",-30%,"27,99€",1,0,1,"15 Oct, 2019",Fri Nov 27 12:21:30 2020
122,Kingdom Come: Deliverance,9,"29,99€",-50%,"14,99€",1,0,0,"13 Feb, 2018",Fri Nov 27 12:21:30 2020
123,Gears 5,6,"39,99€",-75%,"9,99€",1,0,0,"9 Sep, 2019",Fri Nov 27 12:21:30 2020


#### Hur ser våran data ut?

In [6]:
# Här ser vi hur dataframen ser ut om vi inte kör flatten på thelist
pd.DataFrame(thelist).describe()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
count,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
unique,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
top,"{'Name': 'PLAYERUNKNOWN'S BATTLEGROUNDS', 'Sco...","{'Name': 'Assetto Corsa Competizione', 'Score'...","{'Name': 'Total War: WARHAMMER', 'Score': 'Mos...",{'Name': 'Sekiro™: Shadows Die Twice - GOTY Ed...,"{'Name': 'Assassin's Creed® Origins', 'Score':...","{'Name': 'Wasteland 3', 'Score': 'Very Positiv...","{'Name': 'EA STAR WARS™ TRIPLE BUNDLE', 'Score...","{'Name': 'Fallout 4', 'Score': 'Mostly Positiv...","{'Name': 'Europa Universalis IV', 'Score': 'Ve...","{'Name': 'Need for Speed™ Heat', 'Score': 'Ver...",...,"{'Name': 'The Forest', 'Score': 'Very Positive...","{'Name': 'HITMAN™ 2', 'Score': 'Very Positive ...","{'Name': 'F1® 2020', 'Score': 'Very Positive 9...","{'Name': 'Satisfactory', 'Score': 'Overwhelmin...","{'Name': 'Tabletop Simulator', 'Score': 'Overw...","{'Name': 'Arma 3', 'Score': 'Very Positive 90%...","{'Name': 'ARK: Survival Evolved', 'Score': 'Mo...","{'Name': 'Assetto Corsa Ultimate Edition', 'Sc...","{'Name': 'Fall Guys: Ultimate Knockout', 'Scor...","{'Name': 'Black Desert Online', 'Score': 'Most..."
freq,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [7]:
# Här ser vi hur dataframen ser ut med flatten på thelist
pd.DataFrame(flatten(thelist)).describe()

,Name,Score,Original Price,Discount,Discounted Price,Win,Linux,Mac,Release Date,Time
count,125,125,125,125,125,125,125,125,125,125
unique,124,125,32,25,61,1,2,2,106,5
top,Shadow of the Tomb Raider: Definitive Edition,"Very Positive 81% of the 9,704 user reviews fo...","39,99€",-50%,"14,99€",1,,,,Fri Nov 27 12:21:26 2020
freq,2,1,27,18,12,125,99,95,10,25


In [8]:
theFrame.head(10)

,Name,Score,Original Price,Discount,Discounted Price,Win,Linux,Mac,Release Date,Time
0,Counter-Strike: Global Offensive,9,Free to Play,,Free to Play,1,1,1,"21 Aug, 2012",Fri Nov 27 12:21:26 2020
1,DOOM Eternal,9,"59,99€",-67%,"19,79€",1,0,0,"19 Mar, 2020",Fri Nov 27 12:21:26 2020
2,The Sims™ 4,9,"39,99€",-88%,"4,79€",1,0,0,"18 Jun, 2020",Fri Nov 27 12:21:26 2020
3,Borderlands 3,9,"59,99€",-67%,"19,79€",1,0,0,"13 Mar, 2020",Fri Nov 27 12:21:26 2020
4,Red Dead Redemption 2,9,"59,99€",-33%,"40,19€",1,0,0,"5 Dec, 2019",Fri Nov 27 12:21:26 2020
5,Sid Meier’s Civilization® VI,9,"59,99€",-75%,"14,99€",1,1,1,"20 Oct, 2016",Fri Nov 27 12:21:26 2020
6,Halo: The Master Chief Collection,9,"39,99€",-35%,"25,99€",1,0,0,"3 Dec, 2019",Fri Nov 27 12:21:26 2020
7,Total War: WARHAMMER II,9,"59,99€",-66%,"20,39€",1,1,1,"28 Sep, 2017",Fri Nov 27 12:21:26 2020
8,DEATH STRANDING,9,"59,99€",-50%,"29,99€",1,0,0,"14 Jul, 2020",Fri Nov 27 12:21:26 2020
9,Control Ultimate Edition,9,"39,99€",-50%,"19,99€",1,0,0,"27 Aug, 2020",Fri Nov 27 12:21:26 2020


In [9]:
theFrame.tail(10)

,Name,Score,Original Price,Discount,Discounted Price,Win,Linux,Mac,Release Date,Time
115,PAYDAY 2,9,"9,99€",-50%,"4,99€",1,1,0,"13 Aug, 2013",Fri Nov 27 12:21:30 2020
116,Borderlands 3 Ultimate Edition,9,"114,97€",-35%,"74,67€",1,0,0,,Fri Nov 27 12:21:30 2020
117,Medieval Dynasty,9,"24,99€",-20%,"19,99€",1,0,0,"17 Sep, 2020",Fri Nov 27 12:21:30 2020
118,Wolfenstein: Youngblood,6,"29,99€",-70%,"8,99€",1,0,0,"25 Jul, 2019",Fri Nov 27 12:21:30 2020
119,Tom Clancy's Ghost Recon® Wildlands,7,"49,99€",-70%,"14,99€",1,0,0,"6 Mar, 2017",Fri Nov 27 12:21:30 2020
120,Remnant: From the Ashes,9,"39,99€",-50%,"19,99€",1,0,0,"19 Aug, 2019",Fri Nov 27 12:21:30 2020
121,Disco Elysium,9,"39,99€",-30%,"27,99€",1,0,1,"15 Oct, 2019",Fri Nov 27 12:21:30 2020
122,Kingdom Come: Deliverance,9,"29,99€",-50%,"14,99€",1,0,0,"13 Feb, 2018",Fri Nov 27 12:21:30 2020
123,Gears 5,6,"39,99€",-75%,"9,99€",1,0,0,"9 Sep, 2019",Fri Nov 27 12:21:30 2020
124,7 Days to Die,9,"22,99€",-66%,"7,81€",1,1,1,"13 Dec, 2013",Fri Nov 27 12:21:30 2020


In [10]:
theFrame.dtypes

Name                object
Score               object
Original Price      object
Discount            object
Discounted Price    object
Win                  int64
Linux                int64
Mac                  int64
Release Date        object
Time                object
dtype: object

#### Det ser ju bra ut!

## Reflektion
- Jag tyckte väldigt mycket om denhär uppgiften! Jag börjar bli ganska van med python under projektets gång
- Jag använde inte så mycket DataFrames och Series men jag tycker om att arbeta med listor och map/filter funktioner. 